In [ ]:
import pandas as pd

In [ ]:
input_path = '../tmp-data/ss_strategy_1min_7d_avg_adr_range_bars.b1'
known_false_signals = False

In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


class RangeBarStrategy(Strategy):

    per_trade_risk = 0.1
    known_false_signals = False

    def init(self):
       super().init()
       self.false_buys = 0
       self.false_sells = 0


    def bb_upper_near(self, index, close, data):
        try:
            upper_series = data.df.iloc[:index+1]['bb_upper']
            if len(upper_series) < 2:
                return False
            # print(f'upper_series:\n{str(upper_series)}')
            since = self.iterations_back_till_condition(upper_series, lambda x: x >= close)
            # print(f'bb_upper_near(upper_series >= close): index: {index}, close: {close}, since: {since}')
            return since < 2
        except Exception as e:
            print(f'bb_upper_near: exception: {e.__cause__}')
            raise e
        

    def bb_lower_near(self, index, close, data):
       try:
         lower_series = data.df.iloc[:index+1]['bb_lower']
         if len(lower_series) < 2:
                return False
        #  print(f'lower_series:\n{str(lower_series)}')
         since = self.iterations_back_till_condition(lower_series, lambda x: x <= close)
        #  print(f'bb_lower_near(lower_series <= close): index: {index}, close: {close}, since: {since}')
         return since < 2
       except Exception as e:
            print(f'bb_lower_near: exception: {e.__cause__}')
            raise e
       
    def iterations_back_till_condition(self, series, condition):
        count = 0
        for value in series[::-1]:
            if condition(value):
                break
            count += 1
        return count
       
    def bb_upper_pointing_up(self, index, data):
        from scipy.stats import linregress
        bb_seg = data.df.iloc[index-3:index+1]['bb_upper']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_upper_pointing_up: seg_len: {seg_len}, slope: {slope}')
                return slope > 0
            except Exception as e:
                print(f'bb_upper_pointing_up: exception: {str(e)}')
        return False

    def bb_lower_pointing_down(self, index, data):
        from scipy.stats import linregress
        bb_seg = data.df.iloc[index-3:index+1]['bb_lower']
        # print(f'bb_seg: {len(bb_seg)}')
        if len(bb_seg) > 0:
            seg_len = len(bb_seg)
            try:
                slope, _, _, _, _ = linregress(range(seg_len), bb_seg)
                # print(f'bb_lower_pointing_down: seg_len: {seg_len}, slope: {slope}')
                return slope < 0
            except Exception as e:
                print(f'bb_lower_pointing_down: exception: {str(e)}')  
           
        return False
        
    def next(self):
        current_close = self.data.Close[-1]
        index = self.data.index[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        is_long_rsi = self.data.rsi[-1] > 70
        is_long_macd = self.data.macd[-1] > self.data.macd_signal[-1] > 0
        is_bb_upper_near = self.bb_upper_near(index, current_close, self.data)
        is_bb_upper_pointing_up = self.bb_upper_pointing_up(index, self.data)

        is_short_rsi = self.data.rsi[-1] < 30
        is_short_macd = self.data.macd[-1] < self.data.macd_signal[-1] < 0
        is_bb_lower_near = self.bb_lower_near(index, current_close, self.data)
        is_bb_lower_pointing_down = self.bb_lower_pointing_down(index, self.data)
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]

        known_false_pass = True
        if self.known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if is_long_rsi and is_long_macd and is_bb_upper_near and is_bb_upper_pointing_up and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif is_short_rsi and is_short_macd and is_bb_lower_near and is_bb_lower_pointing_down and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size, tp=current_close - potential_profit)
     


In [ ]:
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


class RangeBarStrategy2(Strategy):

    per_trade_risk = 0.1
    known_false_signals = False

    def init(self):
       super().init()
       self.false_buys = 0
       self.false_sells = 0

    def next(self):
        current_close = self.data.Close[-1]
        index = self.data.index[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]

        known_false_pass = True
        if self.known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')    
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            self.sell(size=self.per_trade_risk, sl=current_close + range_size, tp=current_close - potential_profit)
     


## known false signals

| Return % | risk po | start | end | false b | false s | tot trades | max dd | win rate |
| -------- | ------- | ----- | --- | ------- | ------- | ---------- | ------ | -------- |
| 12,537.59 | 0.1 | 2022-03-21 23:10:00 | 2022-07-02 21:02:0 | 0 | 0 | 4796.0 | -18.252684 | 49.582986 |

## unknown false signals

| Return % | risk po | start | end | false b | false s | tot trades | max dd | win rate |
| -------- | ------- | ----- | --- | ------- | ------- | ---------- | ------ | -------- |
| 3,590.15 | 0.1 | 2022-03-21 23:10:00 | 2022-07-02 21:02:0 | 451 | 530 | 5773 | -22.111453 | 42.0925 |

In [ ]:
# df_sample = df
# start = df_sample.iloc[0]['timestamp']
# end = df_sample.iloc[-1]['timestamp']
# print(f'start: {start}, end: {end}')

## Periods of negative return

| Return % | start | start idx | end | end idx |
| -------- | ----- | --------- | --- | ------- | 
| -34.11704 | 2022-03-27 12:47:00 | 1500 | 2022-04-12 01:22:00 | 6500 |
| -59.653264 |2022-03-26 07:52:00| 1418 | 2022-04-27 01:25:00 | 10500 |

## Periods of positive return

| Return % | start | start idx | end | end idx |
| -------- | ----- | --------- | --- | ------- | 
| 649.93 | 2022-04-27 01:26:00 | 10500 | 2022-05-31 18:09:00 | 20500 |
| 1231.28 | 2022-05-31 18:09:00 | 20500 | 2022-07-05 12:46:00 | 30500 |
| 1875.42 | 2022-07-05 12:47:00 | 30500 | 2022-08-10 16:41:00 | 40500 |


check assumption that strat1 doesn't work with start index > 0 because precalculated indications are off, by recalc on section

The full chart shows 80% max draw down, need to discover why the strategy fails in these periods and how to opt out of trading. The number is false signals seems to be ~ consistent % of total trades

In [ ]:
bt = Backtest(df, RangeBarStrategy2, cash=100_000, margin=1/20, commission=(0.03 / 100), exclusive_orders=False)
stats = bt.run()
false_buys = stats._strategy.false_buys
false_sells = stats._strategy.false_sells
print(f'false buys: {false_buys}, false sells: {false_sells}')
stats


Sharpe Ratio                          0.01437
Sortino Ratio                    85554.785001
Calmar Ratio                    143047.418449

